In [18]:
import pandas as pd

df = pd.read_csv('../raw/Perfumes_dataset.csv')
df.head(10)

,brand,perfume,type,category,target_audience,longevity
0,dumont,nitro red,edp,Fresh Scent,Male,Strong
1,dumont,nitro pour homme,edp,Fresh Scent,Male,Strong
2,dumont,nitro white,edp,Fresh Scent,Unisex,Strong
3,dumont,nitro blue,edp,Fresh Scent,Unisex,Strong
4,dumont,nitro green,edp,Fresh Scent,Unisex,Strong
5,dumont,nitro platinum,edp,Mass Pleaser,Male,Strong
6,dumont,nitro intense,edp,Woody Spicy,Male,Strong
7,dumont,nitro black,edp,Woody Spicy,Male,Strong
8,dumont,celerio oros,edp,Oriental Vanilla,Unisex,Medium
9,dumont,celerio epic,edp,Woody Aromatic,Male,Medium


In [4]:
# lowercase brand and perfume
df['brand'] = df['brand'].astype(str).str.strip().str.lower()
df['perfume'] = df['perfume'].astype(str).str.strip().str.lower()

# lowercase type
df['type'] = df['type'].astype(str).str.strip().str.lower()

# lowercase category
df['category'] = df['category'].astype(str).str.strip().str.lower()

# rename 'target audience' to 'gender' and normalize
df = df.rename(columns={'target_audience': 'gender'})
df['gender'] = df['gender'].astype(str).str.strip().str.lower()

In [5]:
df['longevity'].unique()

array(['Strong', 'Medium',
       'Medium–Strong :contentReference[oaicite:0]{index=0}',
       'Medium :contentReference[oaicite:1]{index=1}',
       'Strong :contentReference[oaicite:2]{index=2}',
       'Light–Medium :contentReference[oaicite:3]{index=3}',
       'Strong :contentReference[oaicite:4]{index=4}',
       'Medium :contentReference[oaicite:5]{index=5}', 'Light–Medium',
       'Light', 'Very Strong', '6–8 hours', 'Longevity'], dtype=object)

In [10]:
df['longevity'] = df['longevity'].astype(str).str.strip().str.lower()


In [11]:
import re

def normalize_longevity(value):
    value = str(value).strip().lower()

    # 1️⃣ Map numeric-hour values
    match = re.search(r'(\d+)', value)
    if match:
        hours = int(match.group(1))
        if hours <= 3:
            return 'light'
        elif hours <= 6:
            return 'light-medium'
        elif hours <= 8:
            return 'medium'
        elif hours <= 10:
            return 'medium-strong'
        else:
            return 'strong'

    # 2️⃣ Map textual variants to clean buckets
    mapping = {
        'light': 'light',
        'lite-medium': 'light-medium',
        'light-medium': 'light-medium',
        'medium': 'medium',
        'medium-strong': 'medium-strong',
        'strong': 'strong',
        'very strong': 'very strong'
    }

    # fallback to medium if unknown
    return mapping.get(value, 'medium')


In [12]:
df['longevity'] = df['longevity'].apply(normalize_longevity)


In [13]:
df['longevity'].value_counts()


longevity
medium          563
strong          336
light            64
very strong      29
light-medium     12
Name: count, dtype: int64

In [14]:
import os
os.makedirs("../processed", exist_ok=True)

In [15]:
df.to_csv("../processed/perfumes_clean.csv", index=False)

In [21]:
# Rename columns

df = df.rename(columns={'target_audience': 'gender'})
df['gender'] = df['gender'].astype(str).str.strip().str.lower()

df = df.rename(columns={'type': 'concentration'})
df['concentration'] = df['concentration'].astype(str).str.strip().str.lower()

df = df.rename(columns={'perfume': 'name'})
df['name'] = df['name'].astype(str).str.strip().str.lower()

df.head(10)

,brand,name,concentration,category,gender,longevity
0,dumont,nitro red,edp,Fresh Scent,male,Strong
1,dumont,nitro pour homme,edp,Fresh Scent,male,Strong
2,dumont,nitro white,edp,Fresh Scent,unisex,Strong
3,dumont,nitro blue,edp,Fresh Scent,unisex,Strong
4,dumont,nitro green,edp,Fresh Scent,unisex,Strong
5,dumont,nitro platinum,edp,Mass Pleaser,male,Strong
6,dumont,nitro intense,edp,Woody Spicy,male,Strong
7,dumont,nitro black,edp,Woody Spicy,male,Strong
8,dumont,celerio oros,edp,Oriental Vanilla,unisex,Medium
9,dumont,celerio epic,edp,Woody Aromatic,male,Medium


In [22]:
# Get unique brands
unique_brands = df['brand'].dropna().unique()

# Make a DataFrame for brands table
brands_df = pd.DataFrame({
    'id': range(1, len(unique_brands) + 1),  # numeric IDs starting at 1
    'name': unique_brands
})


In [26]:
brand_to_id = dict(zip(brands_df['name'], brands_df['id']))
df['brand_id'] = df['brand'].map(brand_to_id)


KeyError: 'brand'

In [24]:
df = df.drop(columns=['brand'])


In [25]:
# Export brands table
brands_df.to_csv("../processed/dev_brands.csv", index=False)

# Export perfumes table with brand_id
df.to_csv("../processed/dev_perfumes_with_ids.csv", index=False)


In [27]:
df = df.rename(columns={'brand_id': 'house_id'})
df['house_id'] = df['house_id'].astype(str).str.strip().str.lower()
df.head(10)

,name,concentration,category,gender,longevity,house_id
0,nitro red,edp,Fresh Scent,male,Strong,1
1,nitro pour homme,edp,Fresh Scent,male,Strong,1
2,nitro white,edp,Fresh Scent,unisex,Strong,1
3,nitro blue,edp,Fresh Scent,unisex,Strong,1
4,nitro green,edp,Fresh Scent,unisex,Strong,1
5,nitro platinum,edp,Mass Pleaser,male,Strong,1
6,nitro intense,edp,Woody Spicy,male,Strong,1
7,nitro black,edp,Woody Spicy,male,Strong,1
8,celerio oros,edp,Oriental Vanilla,unisex,Medium,1
9,celerio epic,edp,Woody Aromatic,male,Medium,1


In [28]:
# Export brands table
brands_df.to_csv("../processed/dev_brands.csv", index=False)

# Export perfumes table with brand_id
df.to_csv("../processed/dev_perfumes_with_ids.csv", index=False)

In [30]:
brand_table = pd.read_csv("../supabase_exports/perfume_houses_export.csv")  # contains 'id' (uuid) and 'name'
brand_map = dict(zip(brand_table['name'], brand_table['id']))


In [34]:
# Read original CSV
df_orig = pd.read_csv("../raw/Perfumes_dataset.csv")  # or wherever the original is

# Bring back the brand column
df['brand'] = df_orig['brand'].str.strip().str.lower()  # lowercase to match your cleaned style



In [41]:
df['house_id'] = df['brand'].map(brand_map)


In [43]:
df.to_csv("../processed/dev_perfumes_with_ids.csv", index=False)
